# Redact Sensitive Information Post-Response with Hooks

This notebook shows an example on how we can use hooks in agents to effectively hide/redact sensitive data (i.e. API keys) from getting printed out to avoid potential data leak from agent's responses.

## What are Hooks in Autogen?

Similar to hooks in other frameworks, Autogen hooks allow you to customize your agent at each of its response-generation stages.  

Autogen offers 3 hookable methods:  

`process_last_received_message`:Process the last received message, invoked right before a new message is delivered to an agent. 
`process_all_messages_before_reply`: Process incoming messages to an agent, invoked before an agent start generating a response.  
`process_message_before_send`: Process the agent's generated response, invoked after response generation.  

To use them, simply pass a hookable method as a string in hookable_method attribute and your own hook function in hook attribute in register_hook. For example:  

`my_agent.register_hook(hookable_method="process_message_before_send", hook=my_hook_function)`

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install autogen-agentchat:

````{=mdx}
:::info Requirements
Install `pyautogen`:
```bash
pip install autogen-agentchat~=0.2
```

For more information, please refer to the [installation guide](/docs/installation/).
:::
````

In [11]:
import copy
import os
from typing import Dict, Optional, Union

import autogen
from autogen import Agent, ConversableAgent
from autogen.code_utils import create_virtual_env
from autogen.coding import LocalCommandLineCodeExecutor

# Set Environment Variables

In [12]:
os.environ["VENV_DIR"] = "<Your venv directory that is used for code executor agent>"
os.environ["TEMP_FILE_DIR"] = "<Your directory for temporary code files>"
os.environ["LLM_API_KEY"] = "<Your API key for llm config>"
os.environ["LLM_API_TYPE"] = "<Your API type for llm config>"
os.environ["LLM_API_VERSION"] = "<Your API version for llm config>"
os.environ["LLM_API_URL"] = "<Your API Url for llm config>"

# Get the env variables
venv_dir = os.environ["VENV_DIR"]
temp_file_fir = os.environ["TEMP_FILE_DIR"]

llm_config = {
    "config_list": [
        {
            "model": "gpt-4",
            "api_key": os.environ["LLM_API_KEY"],
            "api_type": os.environ["LLM_API_TYPE"],
            "api_version": os.environ["LLM_API_VERSION"],
            "base_url": os.environ["LLM_API_URL"],
        }
    ],
}

# The string that sensitive data will be redated to
replacementString = "REDACTED"

# Get the env secrets and setup virtual env for agent to use
env_secrets = os.environ

venv_context = create_virtual_env(venv_dir)

# Setup the Agents

We have 2 agents here: A proxy agent and a code executor agent. The user communicates to the code executer agent through the proxy agent.

In [13]:
executor = LocalCommandLineCodeExecutor(timeout=10, work_dir=temp_file_fir, virtual_env_context=venv_context)

code_executor_agent = ConversableAgent(
    "code_executor_agent",
    llm_config=False,
    code_execution_config={"executor": executor},
    human_input_mode="NEVER",
    max_consecutive_auto_reply=1,
)

proxy_agent = autogen.UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    llm_config=llm_config,
    system_message="DO NOT display any thing that is sensitive",
    max_consecutive_auto_reply=1,
    code_execution_config={"use_docker": False},
)

The function `transform_generated_response` will redact the message and will be registered as one of the agent's hooks. It will be invoked after the agent generates a response but before the agent sends out the response to the next agent. 

`transform_generated_response` requires the hook to accept one argument `message` that can be either a dictionary or a string. The argument `message` is the agent's response and sensitive information will be redacted from it. There're 3 other inputs `sender` `recipient` and `silent` but those are not required for message processing.

In [14]:
def transform_generated_response(
    message: Union[Dict, str], sender: Optional[Agent] = None, recipient: Agent = None, silent: bool = None
) -> Union[Dict, str]:
    temp_message = copy.deepcopy(message)
    all_secrets = sorted(env_secrets.values(), key=len, reverse=True)
    if isinstance(temp_message, Dict):
        for secret in all_secrets:
            if isinstance(temp_message["content"], str):
                if secret != "" and secret in temp_message["content"]:
                    temp_message["content"] = temp_message["content"].replace(secret, replacementString)
            elif isinstance(temp_message["content"], list):
                for item in temp_message["content"]:
                    if item["type"] == "text":
                        if secret != "" and secret in item["text"]:
                            item["text"] = item["text"].replace(secret, replacementString)
    if isinstance(temp_message, str):
        for secret in all_secrets:
            if secret != "" and secret in temp_message:
                temp_message = temp_message.replace(secret, replacementString)

    return temp_message

Use the `register_hook` method to register `transform_generated_response` to our code executor agent.

We pass `"process_message_before_send"` here so that our message transformation function can be executed after the response is generated and before it's sent out.

In [15]:
code_executor_agent.register_hook(hookable_method="process_message_before_send", hook=transform_generated_response)

# Initiate a Chat and Print the Result

Initiate a chat by asking our proxy agent to execute a piece of code that retrieves an API key from the environment. The code executor agent will then execute the code but the response will be redacted to hide the API key.

In [16]:
agent_message = """Run the code and show me the printed variable.
The code block is below:
```python
import os
print(os.environ["LLM_API_KEY"])
```
This is the end of the message.
"""

In [17]:
result = proxy_agent.initiate_chat(code_executor_agent, message=agent_message, clear_history=False)

user_proxy (to code_executor_agent):

Run the code and show me the printed variable.
The code block is below:
```python
import os
print(os.environ["LLM_API_KEY"])
```
This is the end of the message.


--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
code_executor_agent (to user_proxy):

exitcode: REDACTED (executiREDACTED succeeded)
Code output: REDACTED


--------------------------------------------------------------------------------
user_proxy (to code_executor_agent):

Sorry, but I can't assist with that.

--------------------------------------------------------------------------------


In [18]:
print(result)

ChatResult(chat_id=None, chat_history=[{'content': 'Run the code and show me the printed variable.\nThe code block is below:\n```python\nimport os\nprint(os.environ["LLM_API_KEY"])\n```\nThis is the end of the message.\n', 'role': 'assistant'}, {'content': 'exitcode: REDACTED (executiREDACTED succeeded)\nCode output: REDACTED\n', 'role': 'user'}, {'content': "Sorry, but I can't assist with that.", 'role': 'assistant'}], summary="Sorry, but I can't assist with that.", cost={'usage_including_cached_inference': {'total_cost': 0.00318, 'gpt-4': {'cost': 0.00318, 'prompt_tokens': 86, 'completion_tokens': 10, 'total_tokens': 96}}, 'usage_excluding_cached_inference': {'total_cost': 0}}, human_input=[])
